## Step 0: Latent Dirichlet Allocation ##

LDA is used to classify text in a document to a particular topic. It builds a topic per document model and words per topic model, modeled as Dirichlet distributions. 

* Each document is modeled as a multinomial distribution of topics and each topic is modeled as a multinomial distribution of words.
* LDA assumes that the every chunk of text we feed into it will contain words that are somehow related. Therefore choosing the right corpus of data is crucial. 
* It also assumes documents are produced from a mixture of topics. Those topics then generate words based on their probability distribution. 

## Step 1: Load the dataset

The dataset we'll use is a list of over one million news headlines published over a period of 15 years. We'll start by loading it from the `abcnews-date-text.csv` file.

In [2]:
'''
Load the dataset from the CSV and save it to 'data_text'
'''
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

Let's glance at the dataset:

In [3]:
'''
Get the total number of documents
'''
print(len(documents))

300000


In [4]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


## Step 2: Data Preprocessing ##

We will perform the following steps:

* **Tokenization**: Split the text into sentences and the sentences into words. Lowercase the words and remove punctuation.
* Words that have fewer than 3 characters are removed.
* All **stopwords** are removed.
* Words are **lemmatized** - words in third person are changed to first person and verbs in past and future tenses are changed into present.
* Words are **stemmed** - words are reduced to their root form.


In [7]:
'''
Loading Gensim and nltk libraries
'''

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [8]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
stemmer = SnowballStemmer("english")

In [14]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [15]:
'''
Preview a document after preprocessing
'''
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


Let's now preprocess all the news headlines we have. 

In [16]:
# TODO: preprocess all the headlines, saving the list of results as 'processed_docs'
processed_docs = documents.headline_text.map(lambda x : preprocess(x))

In [17]:
'''
Preview 'processed_docs'
'''
processed_docs[:10]

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

## Step 3.1: Bag of words on the dataset

Now let's create a dictionary from 'processed_docs' containing the number of times a word appears in the training set. To do that, let's pass `processed_docs` to [`gensim.corpora.Dictionary()`](https://radimrehurek.com/gensim/corpora/dictionary.html) and call it '`dictionary`'.

In [19]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

In [20]:
'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


** Gensim filter_extremes **

[`filter_extremes(no_below=5, no_above=0.5, keep_n=100000)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.filter_extremes)

Filter out tokens that appear in

* less than no_below documents (absolute number) or
* more than no_above documents (fraction of total corpus size, not absolute number).
* after (1) and (2), keep only the first keep_n most frequent tokens (or keep all if None).

In [21]:
'''
OPTIONAL STEP
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
# apply dictionary.filter_extremes() with the parameters mentioned above
dictionary.filter_extremes(no_below=15, no_above=0.1)

** Gensim doc2bow **

[`doc2bow(document)`](https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow)

* Convert document (a list of words) into the bag-of-words format = list of (token_id, token_count) 2-tuples. Each word is assumed to be a tokenized and normalized string (either unicode or utf8-encoded). No further preprocessing is done on the words in document; apply tokenization, stemming etc. before calling this method.

In [22]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(v) for k, v in processed_docs.items()]

In [23]:
'''
Preview BOW for our sample preprocessed document
'''
# Here document_num is document number 4310 which we have checked in Step 2
bow_doc_4310 = bow_corpus[document_num]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 71 ("bushfir") appears 1 time.
Word 107 ("help") appears 1 time.
Word 462 ("rain") appears 1 time.
Word 3530 ("dampen") appears 1 time.


## Step 3.2: TF-IDF on our document set ##

While performing TF-IDF on the corpus is not necessary for LDA implemention using the gensim model, it is recemmended. TF-IDF expects a bag-of-words (integer values) training corpus during initialization. During transformation, it will take a vector and return another vector of the same dimensionality.

*Please note: The author of Gensim dictates the standard procedure for LDA to be using the Bag of Words model.*

** TF-IDF stands for "Term Frequency, Inverse Document Frequency".**

* It is a way to score the importance of words (or "terms") in a document based on how frequently they appear across multiple documents.
* If a word appears frequently in a document, it's important. Give the word a high score. But if a word appears in many documents, it's not a unique identifier. Give the word a low score.
* Therefore, common words like "the" and "for", which appear in many documents, will be scaled down. Words that appear frequently in a single document will be scaled up.

In other words:

* TF(w) = `(Number of times term w appears in a document) / (Total number of terms in the document)`.
* IDF(w) = `log_e(Total number of documents / Number of documents with term w in it)`.

** For example **

* Consider a document containing `100` words wherein the word 'tiger' appears 3 times. 
* The term frequency (i.e., tf) for 'tiger' is then: 
    - `TF = (3 / 100) = 0.03`. 

* Now, assume we have `10 million` documents and the word 'tiger' appears in `1000` of these. Then, the inverse document frequency (i.e., idf) is calculated as:
    - `IDF = log(10,000,000 / 1,000) = 4`. 

* Thus, the Tf-idf weight is the product of these quantities: 
    - `TF-IDF = 0.03 * 4 = 0.12`.

In [24]:
'''
Create tf-idf model object using models.TfidfModel on 'bow_corpus' and save it to 'tfidf'
'''
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [25]:
'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
corpus_tfidf = tfidf[bow_corpus]

In [26]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


## Step 4.1: Running LDA using Bag of Words ##

We are going for 10 topics in the document corpus.

** We will be running LDA using all CPU cores to parallelize and speed up model training.**

Some of the parameters we will be tweaking are:

* **num_topics** is the number of requested latent topics to be extracted from the training corpus.
* **id2word** is a mapping from word ids (integers) to words (strings). It is used to determine the vocabulary size, as well as for debugging and topic printing.
* **workers** is the number of extra processes to use for parallelization. Uses all available cores by default.
* **alpha** and **eta** are hyperparameters that affect sparsity of the document-topic (theta) and topic-word (lambda) distributions. We will let these be the default values for now(default value is `1/num_topics`)
    - Alpha is the per document topic distribution.
        * High alpha: Every document has a mixture of all topics(documents appear similar to each other).
        * Low alpha: Every document has a mixture of very few topics

    - Eta is the per topic word distribution.
        * High eta: Each topic has a mixture of most words(topics appear similar to each other).
        * Low eta: Each topic has a mixture of few words.

* ** passes ** is the number of training passes through the corpus. For  example, if the training corpus has 50,000 documents, chunksize is  10,000, passes is 2, then online training is done in 10 updates: 
    * `#1 documents 0-9,999 `
    * `#2 documents 10,000-19,999 `
    * `#3 documents 20,000-29,999 `
    * `#4 documents 30,000-39,999 `
    * `#5 documents 40,000-49,999 `
    * `#6 documents 0-9,999 `
    * `#7 documents 10,000-19,999 `
    * `#8 documents 20,000-29,999 `
    * `#9 documents 30,000-39,999 `
    * `#10 documents 40,000-49,999` 

In [27]:
'''
Train your lda model using gensim.models.LdaMulticore and save it to 'lda_model'
'''

lda_model = gensim.models.LdaMulticore(bow_corpus, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [28]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(topic, idx ))
    print("\n")

Topic: 0.024*"open" + 0.021*"test" + 0.018*"world" + 0.017*"win" + 0.015*"lead" + 0.014*"south" + 0.012*"take" + 0.012*"timor" + 0.011*"strike" + 0.011*"aussi" 
Words: 0


Topic: 0.034*"report" + 0.029*"help" + 0.017*"deal" + 0.017*"urg" + 0.014*"inquiri" + 0.014*"blaze" + 0.012*"firefight" + 0.012*"close" + 0.011*"bushfir" + 0.011*"river" 
Words: 1


Topic: 0.038*"crash" + 0.022*"closer" + 0.017*"die" + 0.017*"road" + 0.016*"coast" + 0.014*"train" + 0.013*"dead" + 0.012*"kill" + 0.012*"gold" + 0.012*"north" 
Words: 2


Topic: 0.041*"plan" + 0.035*"council" + 0.031*"govt" + 0.028*"water" + 0.017*"urg" + 0.016*"group" + 0.014*"fund" + 0.012*"chang" + 0.012*"concern" + 0.012*"consid" 
Words: 3


Topic: 0.023*"hospit" + 0.022*"labor" + 0.019*"defend" + 0.019*"elect" + 0.016*"protest" + 0.015*"minist" + 0.014*"work" + 0.013*"govt" + 0.013*"chief" + 0.013*"begin" 
Words: 4


Topic: 0.045*"warn" + 0.019*"fight" + 0.017*"england" + 0.016*"year" + 0.015*"nuclear" + 0.014*"threat" + 0.014*"acti

## Step 4.2 Running LDA using TF-IDF ##

In [29]:
'''
Define lda model using corpus_tfidf, again using gensim.models.LdaMulticore()
'''
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                       num_topics=10, 
                                       id2word = dictionary, 
                                       passes = 2, 
                                       workers=2)

In [30]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.026*"charg" + 0.023*"polic" + 0.017*"court" + 0.015*"murder" + 0.014*"jail" + 0.012*"face" + 0.011*"assault" + 0.011*"arrest" + 0.010*"investig" + 0.010*"drug"


Topic: 1 Word: 0.010*"drought" + 0.008*"farmer" + 0.007*"damag" + 0.007*"rain" + 0.007*"indonesian" + 0.006*"fish" + 0.006*"flood" + 0.006*"illeg" + 0.006*"bring" + 0.005*"rat"


Topic: 2 Word: 0.017*"blaze" + 0.013*"firefight" + 0.010*"climat" + 0.008*"crew" + 0.007*"reach" + 0.007*"qanta" + 0.007*"battl" + 0.007*"withdraw" + 0.006*"polit" + 0.006*"contain"


Topic: 3 Word: 0.011*"price" + 0.010*"drink" + 0.008*"doubt" + 0.007*"clean" + 0.007*"lebanon" + 0.007*"quit" + 0.007*"worri" + 0.006*"sell" + 0.006*"lake" + 0.006*"black"


Topic: 4 Word: 0.010*"gold" + 0.008*"solomon" + 0.008*"bodi" + 0.007*"indigen" + 0.007*"coast" + 0.007*"iemma" + 0.007*"fiji" + 0.006*"beach" + 0.006*"closur" + 0.006*"news"


Topic: 5 Word: 0.013*"rudd" + 0.010*"farm" + 0.008*"shortag" + 0.008*"sentenc" + 0.007*"rural" + 0.007*"wind

## Step 5.1: Performance evaluation by classifying sample document using LDA Bag of Words model##

We will check to see where our test document would be classified. 

In [37]:
documents.loc[4310]['headline_text']

'rain helps dampen bushfires'

In [37]:
'''
Text of sample document 4310
'''
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [38]:
'''
Check which topic our test document belongs to using the LDA Bag of Words model.
'''
document_num = 4310
# Our test document is document number 4310

# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.619939923286438	 
Topic: 0.034*"report" + 0.029*"help" + 0.017*"deal" + 0.017*"urg" + 0.014*"inquiri" + 0.014*"blaze" + 0.012*"firefight" + 0.012*"close" + 0.011*"bushfir" + 0.011*"river"

Score: 0.22004538774490356	 
Topic: 0.019*"school" + 0.018*"farmer" + 0.018*"drought" + 0.016*"price" + 0.014*"rise" + 0.014*"market" + 0.013*"fund" + 0.013*"boost" + 0.012*"rain" + 0.012*"feder"

Score: 0.020001837983727455	 
Topic: 0.024*"open" + 0.021*"test" + 0.018*"world" + 0.017*"win" + 0.015*"lead" + 0.014*"south" + 0.012*"take" + 0.012*"timor" + 0.011*"strike" + 0.011*"aussi"

Score: 0.020001837983727455	 
Topic: 0.038*"crash" + 0.022*"closer" + 0.017*"die" + 0.017*"road" + 0.016*"coast" + 0.014*"train" + 0.013*"dead" + 0.012*"kill" + 0.012*"gold" + 0.012*"north"

Score: 0.020001837983727455	 
Topic: 0.041*"plan" + 0.035*"council" + 0.031*"govt" + 0.028*"water" + 0.017*"urg" + 0.016*"group" + 0.014*"fund" + 0.012*"chang" + 0.012*"concern" + 0.012*"consid"

Score: 0.02000183798372745

### It has the highest probability (0.61) to be  part of the topic related to bushfires

## Step 5.2: Performance evaluation by classifying sample document using LDA TF-IDF model##

In [39]:
'''
Check which topic our test document belongs to using the LDA TF-IDF model.
'''
# Our test document is document number 4310
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5715880990028381	 
Topic: 0.010*"drought" + 0.008*"farmer" + 0.007*"damag" + 0.007*"rain" + 0.007*"indonesian" + 0.006*"fish" + 0.006*"flood" + 0.006*"illeg" + 0.006*"bring" + 0.005*"rat"

Score: 0.2683572769165039	 
Topic: 0.026*"charg" + 0.023*"polic" + 0.017*"court" + 0.015*"murder" + 0.014*"jail" + 0.012*"face" + 0.011*"assault" + 0.011*"arrest" + 0.010*"investig" + 0.010*"drug"

Score: 0.02000804804265499	 
Topic: 0.017*"blaze" + 0.013*"firefight" + 0.010*"climat" + 0.008*"crew" + 0.007*"reach" + 0.007*"qanta" + 0.007*"battl" + 0.007*"withdraw" + 0.006*"polit" + 0.006*"contain"

Score: 0.020007845014333725	 
Topic: 0.024*"closer" + 0.015*"govt" + 0.012*"water" + 0.012*"plan" + 0.012*"council" + 0.010*"fund" + 0.009*"urg" + 0.008*"opposit" + 0.007*"group" + 0.006*"reject"

Score: 0.020006971433758736	 
Topic: 0.013*"rudd" + 0.010*"farm" + 0.008*"shortag" + 0.008*"sentenc" + 0.007*"rural" + 0.007*"wind" + 0.006*"violenc" + 0.006*"south" + 0.006*"speed" + 0.006*"revamp"

Sc

## Step 6: Testing model on unseen document ##

In [42]:
unseen_document = "My favorite sports activities are running and swimming."

# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.42000216245651245	 Topic: 0.024*"open" + 0.021*"test" + 0.018*"world" + 0.017*"win" + 0.015*"lead"
Score: 0.22001101076602936	 Topic: 0.028*"iraq" + 0.018*"talk" + 0.016*"australia" + 0.015*"troop" + 0.012*"storm"
Score: 0.21994002163410187	 Topic: 0.023*"hospit" + 0.022*"labor" + 0.019*"defend" + 0.019*"elect" + 0.016*"protest"
Score: 0.02000878006219864	 Topic: 0.045*"warn" + 0.019*"fight" + 0.017*"england" + 0.016*"year" + 0.015*"nuclear"
Score: 0.020008543506264687	 Topic: 0.041*"plan" + 0.035*"council" + 0.031*"govt" + 0.028*"water" + 0.017*"urg"
Score: 0.020007207989692688	 Topic: 0.019*"school" + 0.018*"farmer" + 0.018*"drought" + 0.016*"price" + 0.014*"rise"
Score: 0.020005645230412483	 Topic: 0.038*"crash" + 0.022*"closer" + 0.017*"die" + 0.017*"road" + 0.016*"coast"
Score: 0.02000560238957405	 Topic: 0.034*"report" + 0.029*"help" + 0.017*"deal" + 0.017*"urg" + 0.014*"inquiri"
Score: 0.020005488768219948	 Topic: 0.073*"polic" + 0.031*"charg" + 0.027*"court" + 0.023*"f